In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
# set up working directory
import os
os.chdir('/Users/gaetanpinon/code_projects/hackathon_BioxAI/biohack')
# check working directory
os.getcwd()
print(os.getcwd())


# load dataset commercial_nac65.csv and nacc_processed.csv from data/
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from config import instrumental_variables, confounders, outcome_predictors, outcome, covariates

# load dataset
# df = pd.read_csv('data/nacc/commercial_nacc65.csv')
df = pd.read_csv('data/nacc/nacc_processed.csv')
# show everything when printing
pd.set_option('display.max_columns', None)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

'/Users/gaetanpinon/code_projects/hackathon_BioxAI/biohack'

/Users/gaetanpinon/code_projects/hackathon_BioxAI/biohack


/var/folders/d7/k100c7y9141ckwg6_stkvkf40000gn/T/ipykernel_41204/4133697706.py:20: DtypeWarning: Columns (2,42,98) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/nacc/nacc_processed.csv')


# Processing

In [ ]:
# # remove all columnn with _Missing
# df = df.loc[:, ~df.columns.str.contains('_Missing')]
# rename columns
df.rename(columns={'VISIT': 'AGE_AT_VISIT'}, inplace=True)
df.rename(columns={outcome: 'OUTCOME'}, inplace=True)
# keep only columns in instrumental_variables, confounders, outcome_predictors, and outcome
columns_to_keep = ["NACCID", 'OUTCOME'] + instrumental_variables + confounders + outcome_predictors #+ covariates
df = df[columns_to_keep]

In [ ]:
# # show the distinct values and their counts for columns covariates
# for col in covariates:
#     # print(col)
#     # set "Missing" values to 0
#     df[col] = df[col].replace('Missing', 0)
#     # transform to float
#     df[col] = df[col].astype(float)
#     # print(df[col].value_counts())

In [17]:
# ----------------- #
# CBSTROKE HYPERTEN EPILEP INLIVWTH CVDCOG NACCNIHR TBI DEPOTHR NACCFAM ALCDEM PD ALSFIND PTSD BIPOLAR SCHIZ ANXIETY ALCOHOL NPSYDEV DELIR PTSDDX COGMODE DECCLIN
strings_list = [
    "CBSTROKE", "HYPERTEN", "EPILEP", "INLIVWTH", "CVDCOG", "NACCNIHR", "TBI", 
    "DEPOTHR", "NACCFAM", "ALCDEM", "PD", "ALSFIND", "PTSD", "BIPOLAR", 
    "SCHIZ", "ANXIETY", "ALCOHOL", "NPSYDEV", "DELIR", "PTSDDX", "COGMODE", 
    "DECCLIN",
]
# set all the "Missing" in these columns to 0
df[strings_list] = df[strings_list].replace('Missing', 0)
# convert all the columns in strings_list to int
df[strings_list] = df[strings_list].astype(float)

# ----------------- #
# Step 1: Calculate FOLLOW_UP
df['FOLLOW_UP'] = df.groupby('NACCID')['AGE_AT_VISIT'].transform(lambda x: x.max() - x.min())

# Step 2: Filter out patients with FOLLOW_UP <= 3 years
df = df[df['FOLLOW_UP'] >= 3]

# Step 3: Add AGE_AT_FIRST_VISIT
df['AGE_AT_FIRST_VISIT'] = df.groupby('NACCID')['AGE_AT_VISIT'].transform('min')

# display the first 5 rows of the resulting DataFrame where AGE_AT_VISIT is equal to AGE_AT_FIRST_VISIT
display(df[df['AGE_AT_VISIT'] == df['AGE_AT_FIRST_VISIT']].head())

# create df_after_3_years where AGE_AT_VISIT - AGE_AT_FIRST_VISIT is >=3 and drop duplicates on NACCID
df_3years = df[df['AGE_AT_VISIT'] - df['AGE_AT_FIRST_VISIT'] >= 3]
# create new column AGE_AT_FIRST_VISIT_AFTER_3_YEARS which is the minimum AGE_AT_VISIT for each NACCID in df_after_3_years
df_3years['AGE_AT_FIRST_VISIT_AFTER_3_YEARS'] = df_3years.groupby('NACCID')['AGE_AT_VISIT'].transform('min')
# create df_3years where AGE_AT_VISIT is equal to AGE_AT_FIRST_VISIT_AFTER_3_YEARS and drop duplicates on NACCID
df_3years = df_3years[df_3years['AGE_AT_VISIT'] == df_3years['AGE_AT_FIRST_VISIT_AFTER_3_YEARS'].copy()].drop_duplicates(subset='NACCID')
# rename outcome column to OUTCOME_AFTER_3_YEARSn and keep only columns NACCID and OUTCOME_AFTER_3_YEARS
df_3years.rename(columns={'OUTCOME': 'OUTCOME_AFTER_3_YEARS'}, inplace=True)
df_3years = df_3years[['NACCID', 'OUTCOME_AFTER_3_YEARS']]

# Join df with df_3years on NACCID
df = df.merge(df_3years, on='NACCID', how='left')
# only keep rows where AGE_AT_VISIT is equal to AGE_AT_FIRST_VISIT and drop duplicates on NACCID
df = df[df['AGE_AT_VISIT'] == df['AGE_AT_FIRST_VISIT']].drop_duplicates(subset='NACCID')
# rename OUTCOME to OUTCOME_AT_BASELINE
df.rename(columns={'OUTCOME': 'OUTCOME_AT_BASELINE'}, inplace=True)
# create a new column OUTCOME which is the difference between OUTCOME_AFTER_3_YEARS and OUTCOME_AT_BASELINE
df['OUTCOME'] = df['OUTCOME_AFTER_3_YEARS'] - df['OUTCOME_AT_BASELINE']

df = df[columns_to_keep]

# Display resulting DataFrame
display(df.head())

,NACCID,OUTCOME,AGE_AT_VISIT,INDEPEND,INLIVWTH,NACCAC,NACCBETA,NACCADEP,NACCPDMD,CVDCOG,NACCMOCA,NACCMMSE,FAQ,NACCGDS,CDRGLOB,NACCNIHR,SEX,EDUC,NACCBMI,TBI,NACCAPSY,DEPOTHR,NACCFAM,NACCFFTD,MEDS,ALCDEM,PD,ALSFIND,PTSD,BIPOLAR,SCHIZ,ANXIETY,ALCOHOL,NPSYDEV,DELIR,PTSDDX,COGMODE,MOMODE,JUDGMENT,COMPORT,CDRLANG,DECCLIN,SMOKYRS,CVHATT,CBSTROKE,CBTIA,DIABETES,HYPERTEN,HYPERCHO,B12DEF,THYROID,ARTHRIT,ABUSOTHR,OCD,BPSYS,VISION,HEARING,SOMATIC,EMOT,HXHYPER,FOCLSIGN,HACHIN,CVDIMAG1,PDNORMAL,MEMORY,ORIENT,DELSEV,HALLSEV,AGITSEV,DEPDSEV,ANXSEV,ELATSEV,APASEV,DISNSEV,IRRSEV,MOTSEV,NITESEV,APPSEV,NORMEXAM,COGSTAT,NACCUDSD,AMYLCSF,CSFTAU,EPILEP,SEIZURES,HIV,OTHCOG,DEP,BIPOLDX,SCHIZOP,ANXIET,OTHPSY,DYSILL,SLEEP,FOLLOW_UP,AGE_AT_FIRST_VISIT
38,NACC120323,3.0,80.243379,1.0,1.0,0,1,0,0,0.0,20.000000,26.635739,1.4,1.0,0.5,1.0,1,16.0,29.500000,0.0,0,0.0,1.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.5,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,181.000000,0.0,1.0,0.0,0.0,0.0,0.0,-0.890432,0.0,0.0,1.0,0.5,0.0,0.0,0.0,1.0,1.0,0.0,0.00000,1.0,1.0,0.0,0.0,0.0,1.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.208904,80.243379
53,NACC141167,0.0,67.508219,1.0,1.0,0,0,0,0,0.0,28.000000,21.379281,0.0,0.0,0.0,1.0,1,12.0,23.300000,0.0,0,0.0,1.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,139.000000,0.0,1.0,0.0,0.0,0.0,0.0,1.225082,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.221461,67.508219
113,NACC232927,9.0,76.694064,3.0,1.0,0,0,0,0,0.0,17.000000,17.499384,3.0,5.0,1.0,1.0,0,16.0,27.100000,0.0,0,1.0,1.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.000000,0.000000,0.0,12.069353,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,119.000000,1.0,1.0,0.0,0.0,0.0,0.0,1.426606,0.0,0.0,2.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.29945,0.0,0.0,0.0,0.0,1.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.277397,76.694064
451,NACC817818,3.0,74.196804,1.0,1.0,1,0,1,1,0.0,23.702048,33.480308,2.0,1.0,0.5,1.0,0,16.0,25.100000,0.0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,1.0,0.5,0.000000,0.000000,0.0,-0.003663,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,123.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.742999,0.0,0.0,1.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,1.00000,0.0,0.0,0.0,0.0,2.0,1.0,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.066895,74.196804
548,NACC000385,5.0,80.685845,1.0,1.0,1,0,0,0,0.0,15.886937,24.000000,1.1,1.0,1.0,1.0,0,16.0,25.802091,0.0,0,0.0,1.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.743914,0.214128,1.0,15.000000,2.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,124.948764,0.0,0.0,0.0,0.0,1.0,0.0,1.000000,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.244521,80.685845


/var/folders/d7/k100c7y9141ckwg6_stkvkf40000gn/T/ipykernel_41204/1707245058.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_3years['AGE_AT_FIRST_VISIT_AFTER_3_YEARS'] = df_3years.groupby('NACCID')['AGE_AT_VISIT'].transform('min')


,NACCID,OUTCOME,AGE_AT_VISIT,INDEPEND,INLIVWTH,NACCAC,NACCBETA,NACCADEP,NACCPDMD,CVDCOG,NACCMOCA,NACCMMSE,FAQ,NACCGDS,CDRGLOB,NACCNIHR,SEX,EDUC,NACCBMI,TBI,NACCAPSY,DEPOTHR,NACCFAM,NACCFFTD,MEDS,ALCDEM,PD,ALSFIND,PTSD,BIPOLAR,SCHIZ,ANXIETY,ALCOHOL,NPSYDEV,DELIR,PTSDDX,COGMODE,MOMODE,JUDGMENT,COMPORT,CDRLANG,DECCLIN,SMOKYRS,CVHATT,CBSTROKE,CBTIA,DIABETES,HYPERTEN,HYPERCHO,B12DEF,THYROID,ARTHRIT,ABUSOTHR,OCD,BPSYS,VISION,HEARING,SOMATIC,EMOT,HXHYPER,FOCLSIGN,HACHIN,CVDIMAG1,PDNORMAL,MEMORY,ORIENT,DELSEV,HALLSEV,AGITSEV,DEPDSEV,ANXSEV,ELATSEV,APASEV,DISNSEV,IRRSEV,MOTSEV,NITESEV,APPSEV,NORMEXAM,COGSTAT,NACCUDSD,AMYLCSF,CSFTAU,EPILEP,SEIZURES,HIV,OTHCOG,DEP,BIPOLDX,SCHIZOP,ANXIET,OTHPSY,DYSILL,SLEEP
0,NACC120323,11.0,80.243379,1.0,1.0,0,1,0,0,0.0,20.000000,26.635739,1.4,1.0,0.5,1.0,1,16.0,29.500000,0.0,0,0.0,1.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.5,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,181.000000,0.0,1.0,0.0,0.0,0.0,0.0,-0.890432,0.0,0.0,1.0,0.5,0.0,0.0,0.0,1.0,1.0,0.0,0.00000,1.0,1.0,0.0,0.0,0.0,1.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,NACC141167,0.0,67.508219,1.0,1.0,0,0,0,0,0.0,28.000000,21.379281,0.0,0.0,0.0,1.0,1,12.0,23.300000,0.0,0,0.0,1.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,139.000000,0.0,1.0,0.0,0.0,0.0,0.0,1.225082,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,NACC232927,-1.0,76.694064,3.0,1.0,0,0,0,0,0.0,17.000000,17.499384,3.0,5.0,1.0,1.0,0,16.0,27.100000,0.0,0,1.0,1.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.000000,0.000000,0.0,12.069353,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,119.000000,1.0,1.0,0.0,0.0,0.0,0.0,1.426606,0.0,0.0,2.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.29945,0.0,0.0,0.0,0.0,1.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9,NACC817818,2.5,74.196804,1.0,1.0,1,0,1,1,0.0,23.702048,33.480308,2.0,1.0,0.5,1.0,0,16.0,25.100000,0.0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,1.0,0.5,0.000000,0.000000,0.0,-0.003663,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,123.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.742999,0.0,0.0,1.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,1.00000,0.0,0.0,0.0,0.0,2.0,1.0,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
11,NACC000385,0.0,80.685845,1.0,1.0,1,0,0,0,0.0,15.886937,24.000000,1.1,1.0,1.0,1.0,0,16.0,25.802091,0.0,0,0.0,1.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.743914,0.214128,1.0,15.000000,2.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,124.948764,0.0,0.0,0.0,0.0,1.0,0.0,1.000000,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
# show the distinct values and their counts for CDRGLOB
print(df["CDRGLOB"].value_counts())

CDRGLOB
0.0    9141
0.5    5809
1.0    1680
2.0     341
3.0     133
Name: count, dtype: int64
